# Лабораторная работа №3
## Выполнил студент группы Стеклов М. А. БФИ2001

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
from typing import DefaultDict
from collections import defaultdict
from random import choice
from string import ascii_letters
from time import time
from re import search
from __future__ import annotations
from functools import lru_cache, total_ordering
from heapq import heapify, heappop, heappush

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [2]:
def calc_prefix_func(s: str) -> list[int]:
    n = len(s)
    res = [0] * n
    for i in range(1, n):
        j = res[i - 1]
        while (j > 0 and s[i] != s[j]):
            j = res[j - 1]
        if (s[i] == s[j]):
            j += 1
        res[i] = j
    return res

print(calc_prefix_func("abcaaabc"))

[0, 0, 0, 1, 1, 1, 2, 3]


In [3]:
def find_substring_1(t: str, s: str, reg_sensitivity: bool=True) -> int:
    if not reg_sensitivity:
        t = t.lower()
        s = s.lower()
    n = len(t)
    p = calc_prefix_func(t + '#' + s)
    for i in range(n + 1, len(p)):
        if p[i] == n:
            return i - 2*n
    return -1

#### Упрощенный алгоритм Бойера-Мура

In [4]:
def get_bias_table(s: str) -> DefaultDict[str, int]:
    n = len(s)
    d = defaultdict(lambda: n)
    d[s[-1]] = n
    for i in range(-2, -n - 1, -1):
        if s[i] not in d:
            d[s[i]] = -i - 1
    return d

def find_substring_2(t: str, s: str, reg_sensitivity: bool=True) -> int:
    if not reg_sensitivity:
        t = t.lower()
        s = s.lower()
    bias_table = get_bias_table(t)
    i = len(t) - 1
    while i < len(s):
        for j in range(len(t)):
            if s[i-j] != t[-j-1]:
                i += bias_table[s[i]]
                break
            if j == len(t) - 1:
                return i - len(t) + 1
    return -1

print(find_substring_2("fdasfsad", "adxbgsfgbabbcdaffdasfsaddasdlkfhaslgdfhashdfha"))
            

16


In [5]:
# Тесты
test_str = ''.join([choice(ascii_letters) for i in range(10_000_000)])
test_substr = test_str[9652000:9653500]




In [6]:
t = time()
print(find_substring_1(test_substr, test_str))
print(time() - t)

9652000
2.049683094024658


In [7]:
t = time()
print(find_substring_2(test_substr, test_str))
print(time() - t)

9652000
0.07001972198486328


In [8]:
t = time()
print(search(test_substr, test_str).start())
print(time() - t)

9652000
0.020642518997192383


In [9]:
t = time()
print(test_str.find(test_substr))
print(time() - t)

9652000
0.01182866096496582


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 


In [92]:
test_x = [13, 11, 14, 5, 6, 7, 9, 2, 12, 1, 8, 4, 10, 3, 15, 0] 

In [93]:

def swap(field: tuple[int], a_i: int, b_i: int):
    if a_i < b_i:
        res = (*field[:a_i], field[b_i], *field[a_i+1:b_i], field[a_i], *field[b_i+1:])
    else:
        res = (*field[:b_i], field[a_i], *field[b_i+1:a_i], field[b_i], *field[a_i+1:])
    return res

@lru_cache
def get_dist(a_i: int, b_i: int) -> int:
    return abs(a_i // 4 - b_i // 4) + abs(a_i % 4 - b_i % 4)


@total_ordering
class Position:
    def __init__(self, field: tuple[int],
                 z_i: int, previous: Position = None,
                 changed_number: int = None,
                 path_len : int = 0):
        self.field = field
        self.previous = previous
        self.changed_number = changed_number
        self.path_len = path_len
        self.z_i = z_i
        self.dist = self._calc_all_dists() 
        self.func = self.dist + path_len // 2
    
    def get_positions(self) -> tuple[Position]:
        res: list[Position] = []
        z_i = self.z_i
        if z_i % 4 > 0:
            res.append(Position(swap(self.field, z_i, z_i - 1),
                                z_i - 1,
                                self,
                                self.field[z_i - 1],
                                self.path_len + 1))
        if self.z_i % 4 < 3:
            res.append(Position(swap(self.field, z_i, z_i + 1),
                                z_i + 1,
                                self,
                                self.field[z_i + 1],
                                self.path_len + 1))
        if self.z_i // 4 < 3:
            res.append(Position(swap(self.field, z_i, z_i + 4),
                                z_i + 4,
                                self,
                                self.field[z_i + 4],
                                self.path_len + 1))
        if self.z_i // 4 > 0:
            res.append(Position(swap(self.field, z_i, z_i - 4),
                                z_i - 4,
                                self,
                                self.field[z_i - 4],
                                self.path_len + 1))
        return res
    
    def calc_path(self) -> list[int]:
        if self.previous is None:
            return []
        return self.previous.calc_path() + [self.changed_number]
    
    def _calc_all_dists(self) -> int:
        res = 0
        for i, n in enumerate(self.field):
            if n == 0:
                continue
            res += get_dist(i, n - 1)
            
            """
            if (y := i // 4) == (y_g := (n - 1) // 4):
                if y < y_g:
                    for j in range(y + 1, y_g + 1):
                        if self.field[j] < y:
                            res += 2
                            break
                else:
                    for j in range(y_g, y):
                        if self.field[j] > y:
                            res += 2
                            break
            if (x := i % 4) == (x_g := (n - 1) % 4):
                if x < x_g:
                    for j in range(x + 1, x_g + 1):
                        if self.field[j] < x:
                            res += 2
                            break
                else:
                    for j in range(x_g, x):
                        if self.field[j] > x:
                            res += 2
                            break
            """
            
        return res
    
    def __lt__(self, other: Position):
        return self.func < other.func
    
    def __eq__(self, other: Position):
        return self.func == other.func
    
    def __hash__(self) -> int:
        return hash(self.field)
    
    def __str__(self) -> str:
        str_field = list(map(str, self.field))
        return '\n'.join(' '.join(str_field[i:i+4]) for i in range(0, 15, 4))

    


In [94]:

def task2(x: list[int]) -> list[int]:
    t = x[::]
    t = list(filter(bool, t[:4] + t[4:8][::-1] + t[8:12] + t[12:17][::-1]))
    k = len([(i,j) for i in range(14) for j in range(i, 15) if t[i] > t[j]])
    res = []
    if k % 2 == 0:
        return res
    z_i = x.index(0)
    x = tuple(x)
    opened_positions_list: list[Position] = [Position(x, z_i)]
    heapify(opened_positions_list)
    opened_positions: set[Position] = {opened_positions_list[0]}
    closed_positions: set[Position] = set() 
    min_dist = 9999
    while opened_positions_list[0].dist > 0:
        if opened_positions_list[0].dist < min_dist:
            min_dist = opened_positions_list[0].dist
            print(min_dist)
            print(opened_positions_list[0])
        new_positions = opened_positions_list[0].get_positions()
        opened_positions.remove(opened_positions_list[0])
        closed_positions.add(heappop(opened_positions_list)) 
        for pos in new_positions:
            if pos in closed_positions or pos in opened_positions:
                continue
            heappush(opened_positions_list, pos)
            opened_positions.add(pos)
    return opened_positions_list[0].calc_path()

In [95]:
t = time.time()
print(task2(test_x))
print(time.time() - t)

38
13 11 14 5
6 7 9 2
12 1 8 4
10 3 15 0
37
13 11 14 5
6 7 9 2
12 1 8 4
0 10 3 15
36
13 11 14 5
6 0 7 2
12 1 9 8
10 3 15 4
35
13 11 14 5
0 6 7 2
12 1 9 8
10 3 15 4
34
0 11 14 5
13 6 7 2
12 1 9 8
10 3 15 4
33
13 11 14 5
6 1 7 2
12 3 9 8
0 10 15 4
32
13 14 5 2
6 11 7 0
12 1 9 8
10 3 15 4
31
13 14 5 2
6 11 7 8
12 1 9 0
10 3 15 4
30
13 14 5 2
6 11 7 8
12 1 9 4
10 3 15 0
29
13 14 5 2
6 11 7 8
12 1 9 4
0 10 3 15
28
13 14 5 2
6 0 7 8
1 11 9 4
12 10 3 15
27
13 14 5 2
0 6 7 8
1 11 9 4
12 10 3 15
26
13 14 5 2
1 6 7 8
0 11 9 4
12 10 3 15
25
13 14 5 2
1 6 7 8
12 11 9 4
0 10 3 15
24
13 14 5 2
1 6 7 8
11 9 3 4
12 10 15 0
23
13 14 5 2
1 6 7 8
12 9 3 4
0 10 11 15
22
1 14 2 7
13 6 5 8
12 9 11 4
10 3 15 0
21
1 14 2 7
13 6 5 8
12 9 11 4
0 10 3 15
20
1 14 2 7
13 6 5 8
9 10 11 4
12 0 3 15
19
1 14 2 7
13 6 5 8
9 10 11 4
0 12 3 15
18
1 14 0 2
5 6 7 8
13 12 11 4
10 9 3 15
17
1 14 2 0
5 6 7 8
13 12 11 4
10 9 3 15
16
1 14 2 8
5 6 7 4
13 9 12 11
10 0 3 15
15
1 14 2 8
5 6 7 4
13 9 12 11
0 10 3 15
14
1 14 2 8
5 6 

### Вывод